In [11]:
import pandas as pd
import pandas as pd
import requests
import json

In [5]:
df = pd.read_csv(r"C:\Users\gerar\Documents\omdena-mumbai-chapter-air-quality\weather\historical_weather_data_updated.csv")

In [6]:
df.tail()

,Unnamed: 0,input_latitude,input_longitude,date,temperature,humidity,surface_pressure,rain,weathercode,cloudcover_low,cloudcover_mid,cloudcover_high,wind_speed,wind_direction,windgusts_10m,output_latitude,output_longitude
427147,427147,19.04946,72.923,2023-03-20T19:00,24.3,73,1008.7,0.0,1,13,11,7,3.3,221,10.8,19.099998,73.0
427148,427148,19.04946,72.923,2023-03-20T20:00,24.4,73,1008.7,0.0,1,22,35,1,1.8,90,11.2,19.099998,73.0
427149,427149,19.04946,72.923,2023-03-20T21:00,22.0,84,1007.5,0.1,51,11,47,0,6.1,118,10.1,19.099998,73.0
427150,427150,19.04946,72.923,2023-03-20T22:00,22.1,83,1007.2,0.0,1,9,37,4,5.8,120,11.2,19.099998,73.0
427151,427151,19.04946,72.923,2023-03-20T23:00,22.8,80,1007.2,0.0,1,9,21,7,3.8,139,11.2,19.099998,73.0


last date is 2023 march 20 23:00

need data from march 21 - april 6th

In [7]:
#air quality station coordinates
coordinates = [[19.044, 73.0325],
 [19.1135051, 73.008978],
 [19.053536, 72.84643],
 [19.1375, 72.915056],
 [19.23241, 72.86895],
 [19.1653323, 72.922099],
 [18.96702, 72.84214],
 [18.9936162, 72.8128113],
 [19.10861, 72.83622],
 [19.192056, 72.9585188],
 [19.25292, 73.142019],
 [19.047, 72.8746],
 [19.2058, 72.8682],
 [19.175, 72.9419],
 [19.19709, 72.82204],
 [19.008751, 73.01662],
 [19.000083, 72.813993],
 [19.072830200195, 72.882606506348],
 [19.0863, 72.8888],
 [19.10078, 72.87462],
 [19.2243333, 72.8658113],
 [19.04946, 72.923]]

In [8]:
def get_live_data(latitude, longitude):
  api_url = "https://api.open-meteo.com/v1/forecast?latitude=" + str(latitude) + "&longitude=" + str(longitude) + \
    "&hourly=temperature_2m,relativehumidity_2m,rain,weathercode,surface_pressure," + \
    "cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,windgusts_10m&start_date=2023-03-21&end_date=2023-04-06"

  #accessing the api and pulling information
  headers = {"accept": "application/json"}
  response = requests.get(api_url, headers = headers)
  print(f"status_code: {response.status_code}")

  #collect and store releveant weather metrics for coordinates
  lat = response.json()['latitude']
  lon = response.json()["longitude"]
  hours = response.json()["hourly"]["time"]
  temperature = response.json()["hourly"]["temperature_2m"]
  humidity = response.json()["hourly"]["relativehumidity_2m"]
  pressure = response.json()["hourly"]["surface_pressure"]
  rain = response.json()["hourly"]["rain"]
  weathercode = response.json()["hourly"]["weathercode"]
  cloudcover_low = response.json()["hourly"]["cloudcover_low"]
  cloudcover_mid = response.json()["hourly"]["cloudcover_mid"]
  cloudcover_high = response.json()["hourly"]["cloudcover_high"]
  windspeed = response.json()["hourly"]["windspeed_10m"]
  winddirection = response.json()["hourly"]["winddirection_10m"]
  windgusts_10m = response.json()["hourly"]["windgusts_10m"]
  

  data = {"input_latitude": latitude,
          "input_longitude": longitude,
          "date": hours,
          "temperature": temperature,
          "humidity": humidity,
          "surface_pressure": pressure, 
          "rain": rain,
          "weathercode": weathercode ,
          "cloudcover_low": cloudcover_low,
          "cloudcover_mid": cloudcover_mid,
          "cloudcover_high": cloudcover_high,
          "wind_speed": windspeed,
          "wind_direction": winddirection,
          "windgusts_10m": windgusts_10m,
          "output_latitude": lat,
          "output_longitude": lon
          }
          
  df = pd.DataFrame(data)
  print("DataFrame was succesfully created")
  return df

In [9]:
def get_final_df(coordinates):
  final_df = pd.DataFrame(columns = ["input_latitude", "input_longitude", "date", "temperature", "humidity", "surface_pressure", 
                                     "rain", "weathercode", "cloudcover_low", "cloudcover_mid", "cloudcover_high", "wind_speed", 
                                     "wind_direction", "windgusts_10m", "output_latitude", "output_longitude"])
  for x in range(0, len(coordinates)):
    temp_lat = coordinates[x][0]
    temp_lon = coordinates[x][1]

    temp_df = pd.DataFrame()
    temp_df = get_live_data(temp_lat, temp_lon)
    print(f'Temperary DataFrame Number {x} has been created.')

    final_df = final_df.merge(temp_df, how="outer")
    print(f'df #{x}: Merged to final_df')
    print('------------------------------------------------------------------')
  print("final_df has been completed")
  return final_df

In [12]:
final_df = get_final_df(coordinates)
final_df.head()

status_code: 200
DataFrame was succesfully created
Temperary DataFrame Number 0 has been created.
df #0: Merged to final_df
------------------------------------------------------------------
status_code: 200
DataFrame was succesfully created
Temperary DataFrame Number 1 has been created.
df #1: Merged to final_df
------------------------------------------------------------------
status_code: 200
DataFrame was succesfully created
Temperary DataFrame Number 2 has been created.
df #2: Merged to final_df
------------------------------------------------------------------
status_code: 200
DataFrame was succesfully created
Temperary DataFrame Number 3 has been created.
df #3: Merged to final_df
------------------------------------------------------------------
status_code: 200
DataFrame was succesfully created
Temperary DataFrame Number 4 has been created.
df #4: Merged to final_df
------------------------------------------------------------------
status_code: 200
DataFrame was succesfully cr

,input_latitude,input_longitude,date,temperature,humidity,surface_pressure,rain,weathercode,cloudcover_low,cloudcover_mid,cloudcover_high,wind_speed,wind_direction,windgusts_10m,output_latitude,output_longitude
0,19.044,73.0325,2023-03-21T00:00,23.1,83,1003.8,0.0,3,42,37,93,1.0,315,8.6,19.125,73.0
1,19.044,73.0325,2023-03-21T01:00,22.8,83,1004.6,0.0,3,48,35,100,2.5,8,13.3,19.125,73.0
2,19.044,73.0325,2023-03-21T02:00,23.1,81,1005.4,0.0,3,36,0,100,3.2,27,7.6,19.125,73.0
3,19.044,73.0325,2023-03-21T03:00,24.3,75,1006.4,0.0,3,35,0,100,1.9,338,7.6,19.125,73.0
4,19.044,73.0325,2023-03-21T04:00,25.7,69,1007.0,0.0,2,38,34,23,3.1,249,18.7,19.125,73.0


In [ ]:
df = df.drop(axis=1, columns=["Unnamed: 0"])

In [14]:
complete_df = df.merge(final_df, how="outer")
complete_df.tail()

,input_latitude,input_longitude,date,temperature,humidity,surface_pressure,rain,weathercode,cloudcover_low,cloudcover_mid,cloudcover_high,wind_speed,wind_direction,windgusts_10m,output_latitude,output_longitude
436123,19.04946,72.923,2023-04-06T19:00,27.0,82,1009.5,0.0,2,0,44,0,1.9,202,6.8,19.125,72.875
436124,19.04946,72.923,2023-04-06T20:00,26.9,83,1009.0,0.0,2,0,57,0,0.5,135,4.3,19.125,72.875
436125,19.04946,72.923,2023-04-06T21:00,26.8,85,1008.5,0.0,2,0,69,0,1.8,53,2.2,19.125,72.875
436126,19.04946,72.923,2023-04-06T22:00,26.7,86,1008.4,0.0,2,0,70,0,2.0,45,2.9,19.125,72.875
436127,19.04946,72.923,2023-04-06T23:00,26.6,87,1008.5,0.0,2,0,72,0,2.0,45,3.2,19.125,72.875


In [16]:
complete_df.to_csv("complete_weather_data.csv")